In [6]:
from nga.data.datamanagers.plane_geometry_datamanager import PlaneGeometryDataManagerConfig, PlaneGeometryDataManager
from nerfstudio.engine.trainer import TrainerConfig
from nerfstudio.configs.method_configs import method_configs
from nerfstudio.configs.dataparser_configs import dataparsers as dataparser_configs
from nerfstudio.utils.eval_utils import eval_load_checkpoint
from nerfstudio.cameras.rays import RayBundle
from nerfstudio.utils import colormaps

from PIL import Image
from pathlib import Path
import yaml
import torch

In [7]:
config_path = "outputs/pattern_plane1/tensorf/2023-08-30_135121/config.yml"
with open(config_path, "r") as f:
    config_str = f.read()
config = yaml.load(config_str, Loader=yaml.Loader)
# config.pipeline.datamanager.__class__ = PlaneGeometryDataManagerConfig
# config.pipeline.datamanager._target = PlaneGeometryDataManager
# dataparser = config.pipeline.datamanager.dataparser
# config.pipeline.datamanager = PlaneGeometryDataManagerConfig()
# config.pipeline.datamanager.dataparser = dataparser
# config.pipeline.datamanager.data = config.data
config.print_to_terminal()

──────────────────────────────────────────────────────── Config ────────────────────────────────────────────────────────

TrainerConfig(
    _target=<class 'nerfstudio.engine.trainer.Trainer'>,
    output_dir=PosixPath('outputs'),
    method_name='tensorf',
    experiment_name='pattern_plane1',
    project_name='nerfstudio-project',
    timestamp='2023-08-30_135121',
    machine=MachineConfig(seed=42, num_devices=1, num_machines=1, machine_rank=0, dist_url='auto', device_type='cuda'),
    logging=LoggingConfig(
        relative_log_dir=PosixPath('.'),
        steps_per_log=10,
        max_buffer_size=20,
        local_writer=LocalWriterConfig(
            _target=<class 'nerfstudio.utils.writer.LocalWriter'>,
            enable=True,
            stats_to_track=(
                <EventName.ITER_TRAIN_TIME: 'Train Iter (time)'>,
                <EventName.TRAIN_RAYS_PER_SEC: 'Train Rays / Sec'>,
                <EventName.CURR_TEST_PSNR: 'Test PSNR'>,
                <EventName.VIS_RAYS_PER_SEC: 'Vis Rays / Sec'>,
                <EventName.TEST_RAYS_PER_SEC: 'Test Rays / Sec'>,
                <EventName.ETA: 'ETA (time)'>
            ),
            max_log_size=10
        ),
        profiler='basic'
    ),
    viewer=ViewerConfig(
        relative_log_filename='viewer_log_filename.txt',
        websocket_port=None,
        websocket_port_default=7007,
        websocket_host='0.0.0.0',
        num_rays_per_chunk=32768,
        max_num_display_images=512,
        quit_on_train_completion=True,
        image_format='jpeg',
        jpeg_quality=90
    ),
    pipeline=VanillaPipelineConfig(
        _target=<class 'nerfstudio.pipelines.base_pipeline.VanillaPipeline'>,
        datamanager=VanillaDataManagerConfig(
            _target=<class 'nerfstudio.data.datamanagers.base_datamanager.VanillaDataManager'>,
            data=PosixPath('/home/ccl/Datasets/NeRF/nerf-geometry-analysis/pattern_plane1'),
            camera_optimizer=CameraOptimizerConfig(
                _target=<class 'nerfstudio.cameras.camera_optimizers.CameraOptimizer'>,
                mode='off',
                position_noise_std=0.0,
                orientation_noise_std=0.0,
                optimizer=AdamOptimizerConfig(
                    _target=<class 'torch.optim.adam.Adam'>,
                    lr=0.0006,
                    eps=1e-15,
                    max_norm=None,
                    weight_decay=0
                ),
                scheduler=ExponentialDecaySchedulerConfig(
                    _target=<class 'nerfstudio.engine.schedulers.ExponentialDecayScheduler'>,
                    lr_pre_warmup=1e-08,
                    lr_final=None,
                    warmup_steps=0,
                    max_steps=10000,
                    ramp='cosine'
                ),
                param_group='camera_opt'
            ),
            masks_on_gpu=False,
            images_on_gpu=False,
            dataparser=NerfstudioDataParserConfig(
                _target=<class 'nerfstudio.data.dataparsers.nerfstudio_dataparser.Nerfstudio'>,
                data=PosixPath('/home/ccl/Datasets/NeRF/nerf-geometry-analysis/pattern_plane1'),
                scale_factor=1.0,
                downscale_factor=None,
                scene_scale=1.0,
                orientation_method='up',
                center_method='poses',
                auto_scale_poses=True,
                eval_mode='fraction',
                train_split_fraction=0.9,
                eval_interval=8,
                depth_unit_scale_factor=0.001
            ),
            train_num_rays_per_batch=4096,
            train_num_images_to_sample_from=-1,
            train_num_times_to_repeat_images=-1,
            eval_num_rays_per_batch=4096,
            eval_num_images_to_sample_from=-1,
            eval_num_times_to_repeat_images=-1,
            eval_image_indices=(0,),
            collate_fn=<function nerfstudio_collate at 0x7fb449ed4a60>,
            camera_res_scale_factor=1.0,
            patch_size=1,
            pixel_sampler=PixelSamplerConfig(
                _target=<class 'nerfstudio.data.pixel_samplers.Pixel

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:

# load checkpoints from wherever they were saved
# TODO: expose the ability to choose an arbitrary checkpoint
config.load_dir = config.get_checkpoint_dir()

# setup pipeline (which includes the DataManager)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = config.pipeline.setup(device=device, test_mode="val")
pipeline.eval()

[17:02:32] Auto image downscale factor of 1                                                 ]8;id=979702;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=890914;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#349\349]8;;\

            Dataset is overriding train_indices to [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, ]8;id=218871;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=141046;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\
           20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,                              
           40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,                              
           60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,                              
           80, 81, 82, 83, 84, 85, 86, 87, 88, 89]                                                                      

            Dataset is overriding val_indices to [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]   ]8;id=194120;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=647113;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\

Setting up training dataset...

Caching all 80 images.

Output()

Setting up evaluation dataset...

Caching all 10 images.

Output()

VanillaPipeline(
  (datamanager): VanillaDataManager(
    (train_camera_optimizer): CameraOptimizer()
    (train_ray_generator): RayGenerator(
      (pose_optimizer): CameraOptimizer()
    )
    (eval_camera_optimizer): CameraOptimizer()
    (eval_ray_generator): RayGenerator(
      (pose_optimizer): CameraOptimizer()
    )
  )
  (_model): TensoRFModel(
    (collider): AABBBoxCollider()
    (field): TensoRFField(
      (feature_encoding): NeRFEncoding()
      (direction_encoding): NeRFEncoding()
      (density_encoding): TensorVMEncoding()
      (color_encoding): TensorVMEncoding()
      (mlp_head): MLP(
        (activation): ReLU()
        (out_activation): ReLU()
        (layers): ModuleList(
          (0): Linear(in_features=150, out_features=128, bias=True)
          (1): Linear(in_features=128, out_features=128, bias=True)
        )
      )
      (B): Linear(in_features=144, out_features=27, bias=False)
      (field_output_rgb): RGBFieldHead(
        (activation): Sigmoid()
      

In [9]:

checkpoint_path, step = eval_load_checkpoint(config, pipeline)

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from outputs/pattern_plane1/tensorf/2023-08-30_135121/nerfstudio_models/step-000029999.ckpt

In [35]:


output_path = Path("output.json")
render_output_path = Path("outputs/renders")
assert output_path.suffix == ".json"
if render_output_path is not None:
    render_output_path.mkdir(parents=True, exist_ok=True)


dataparser_scale = 0.3384755453003446
def next_eval():
    n = 21
    x = torch.linspace(-0.5, 0.5, n) * dataparser_scale
    y = torch.linspace(-0.5, 0.5, n) * dataparser_scale
    grid_x, grid_y = torch.meshgrid(x, y)
    origins = torch.stack([grid_x, grid_y, torch.ones([n, n])], dim=-1)
    directions = torch.zeros_like(origins)
    directions[:, :, 2] = -1.0
    pixel_area = torch.ones((n, n, 1)) / (n ** 2)
    camera_indices = torch.zeros((n, n, 1))
    ray_bundle = RayBundle(
        origins=origins, directions=directions, pixel_area=pixel_area, camera_indices=camera_indices
    )
    return ray_bundle, {}


camera_ray_bundle, batch = next_eval()
camera_ray_bundle = camera_ray_bundle.to(device)
height, width = camera_ray_bundle.shape
num_rays = height * width
outputs = pipeline.model.get_outputs_for_camera_ray_bundle(camera_ray_bundle)
# metrics_dict, images_dict = pipeline.model.get_image_metrics_and_images(outputs, batch)


rgb = outputs["rgb"]
acc = colormaps.apply_colormap(outputs["accumulation"])
depth = colormaps.apply_depth_colormap(
    outputs["depth"],
    accumulation=outputs["accumulation"],
)
Image.fromarray((rgb * 255).byte().cpu().numpy()).save(
    render_output_path / "rgb.jpg"
)
Image.fromarray((acc * 255).byte().cpu().numpy()).save(
    render_output_path / "acc.jpg"
)
Image.fromarray((depth * 255).byte().cpu().numpy()).save(
    render_output_path / "depth.jpg"
)

In [34]:
dscale = (2.897777557373047 - 0.5209445357322693) / 2
dmean = (2.897777557373047 + 0.5209445357322693) / 2
z = 1 - outputs["depth"]
z /= dataparser_scale
z -= -1.8674187660217285
z

tensor([[[-0.0003],
         [-0.0003],
         [-0.0003],
         ...,
         [-0.0006],
         [-0.0006],
         [-0.0006]],

        [[-0.0003],
         [-0.0003],
         [-0.0003],
         ...,
         [-0.0006],
         [-0.0006],
         [-0.0006]],

        [[-0.0003],
         [-0.0003],
         [-0.0003],
         ...,
         [-0.0006],
         [-0.0006],
         [-0.0006]],

        ...,

        [[-0.0003],
         [-0.0021],
         [-0.0021],
         ...,
         [-0.0011],
         [-0.0011],
         [-0.0012]],

        [[-0.0003],
         [-0.0021],
         [-0.0021],
         ...,
         [-0.0011],
         [-0.0011],
         [-0.0011]],

        [[-0.0003],
         [-0.0003],
         [-0.0021],
         ...,
         [-0.0011],
         [-0.0011],
         [-0.0011]]], device='cuda:0')

In [19]:
pipeline.model.scene_box

SceneBox(aabb=tensor([[-1., -1., -1.],
        [ 1.,  1.,  1.]]))